In [1]:
#load the grouped list for colnames
from utils import *
import json
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime

In [15]:
# from date unix extract year
def extract_year(row):
    date_unix = int(row['date_unix'])
    return datetime.utcfromtimestamp(date_unix).year

# extract siteInfo column
def extract_siteInfo(row, site_info_dict, colname):
    ap_code = row['ap_code']
    
    return site_info_dict[ap_code][colname]


In [35]:
air_pollution = 'no2'
with_o3 = False
csv_path = 'MaryleboneRoad.csv'


if with_o3:
    o3_str = 'with_o3'
else:
    o3_str = 'without_o3'

output_path = air_pollution+'_union_'+o3_str+'_'+csv_path
data_fill_path = 'configs/'+air_pollution+"_unionSites_"+o3_str+"_median.json"
model_path = 'models/'+o3_str+'/'+air_pollution+'_model.txt'

In [4]:
if not with_o3:
    if 'o3' in continuous_variables:
        continuous_variables.remove('o3')
if with_o3:
    if 'o3' not in continuous_variables:
        continuous_variables.append('o3')
    if 'o3' in air_pollutions:
        air_pollutions.remove('o3')

features_names = continuous_variables + category_variables

In [5]:
# Opening JSON file
with open(data_fill_path) as json_file:
    data_fill = json.load(json_file)

In [6]:
model = lgb.Booster(model_file=model_path)

In [7]:
df_csv = pd.read_csv(csv_path)

In [8]:
#add column year
df_csv['year'] = df_csv.apply(lambda row : extract_year(row), axis=1)

#if the ap_code column is called code 
if 'code' in df_csv.columns and 'ap_code' not in df_csv.columns:
    df_csv = df_csv.rename(columns={"code": "ap_code"})

for colname in site_info_columns:
    df_csv[colname] = df_csv.apply(lambda row : extract_siteInfo(row, site_info_dict, colname), axis=1)

In [21]:
for ap_code in np.unique(df_csv['ap_code']):

    for colname in continuous_variables:
        condition = (df_csv['ap_code'] == ap_code)
        df_csv.loc[condition, colname] = df_csv.loc[condition, colname].fillna(data_fill[ap_code][colname])

In [26]:
for c in category_variables:
    if c in df_csv.columns:
        df_csv[c] = df_csv[c].astype('category')

In [28]:
y_predicts = model.predict(df_csv[features_names])

In [36]:
df_csv[air_pollution+'_predict'] = y_predicts
df_csv.to_csv(output_path, index=False)